**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Dropout

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act is designed in order to tackle cleverly the exploration exploitation dilema when choosing the next action a.
At each time, we ever explore or ever exploit the learned policy.

More specifically, the epsilon parameter is the probability at which we explore randomly (uniformly) among the states set $\{s_1,s_2,s_3,s_4\}$.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:] = 256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=201 # set small when debugging
epochs_test=1 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array board contains the bonus and malus positions in the discretized space.

The array position not only does contains the position of the rat, but especially enables to know wether or not there is a border in the $3\times 3$ square around the current position (a border is characterized by a $-1$).

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        state = env.reset()
        game_over = False
        win = 0
        lose = 0
        
        while not game_over:
            action = agent.act(state,train=False)
        
            #prev_state = state
            state, reward, game_over = env.act(action)
            
            if reward > 0:
                win = win + reward
            else:
                lose = lose - reward
        
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 6.5/8.0. Average score (-1.5)
Final score: -1.5


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}







On the one hand, we have :
\begin{align*}
Q^\pi(s,a) &= \mathbb E_{p^\pi}[\sum_{t=0}^{+\infty} \gamma^tr(s_t,a_t)|s_0 = s, a_0 = a]\\
&= r(s,a)+\gamma\mathbb E_{p^\pi}[\sum_{t=1}^{+\infty} \gamma^{t-1}r(s_t,a_t)|s_0 = s, a_0 = a]\\
&= r(s,a)+\gamma\sum_{s',a'}\mathbb P_{p^\pi}(s_1 = s',a_1 = a'|s_0 = s,a_0 = a)\mathbb E_{p^\pi}[\sum_{t=1}^{+\infty} \gamma^{t-1}r(s_t,a_t)|s_0 = s, a_0 = a, s_1 = s', a_1 = a']
\end{align*}

Since \begin{equation} \mathbb E_{p^\pi}[\sum_{t=1}^{+\infty} \gamma^{t-1}r(s_t,a_t)|s_0 = s, a_0 = a, s_1 = s', a_1 = a'] = \mathbb E_{p^\pi}[\sum_{t=1}^{+\infty} \gamma^{t-1}r(s_t,a_t)| s_1 = s', a_1 = a'] = Q^\pi(s',a'),\end{equation}
we have eventually :
\begin{equation} Q^\pi(s,a) = \mathbb E_{(s',a')\sim p(.|s_0 = s, a_0 = a)}[ r(s,a)+\gamma Q^\pi(s',a') ] .  \end{equation}

On the other hand, we may now write for $s$ a state and $a$ an action :

\begin{align*}
Q^*(s,a) &= \underset{\pi}{\max}Q^\pi(s,a)\\
&= r(s,a) + \gamma \underset{\pi}{\max} \mathbb E_{(s',a')\sim p(.|s_0 = s, a_0 = a)}[ Q^\pi(s',a') ].
\end{align*}

For $\pi$ a policy, let's write $\pi$ as $(a_0',\pi')$.
Thus, we have :
\begin{align*}
Q^*(s,a) &= r(s,a) + \gamma \underset{\pi = (a_0',\pi')}{\max} \mathbb E_{(s',a')\sim p(.|s_0 = s, a_0 = a)}[ Q^\pi(s',a') ]\\
&= r(s,a) + \gamma \mathbb E_{s'\sim p(.|s_0 = s, a_0 = a)}[ \underset{a_0'}{\max} Q^{*}(s',a_0') ]\\
&= r(s,a) + \gamma \mathbb E_{s' \sim \pi^*(.|s,a)}[ \underset{a_0'}{\max} Q^{*}(s',a_0') ].
\end{align*}



As a consequence, we would like to minimize the function of $\theta$ given by :

\begin{align*}
\mathcal{L}(\theta) &= \Vert r(s,a) + \gamma \mathbb E_{s' \sim \pi^*(.|s,a)}[ \underset{a_0'}{\max} Q^{*}(s',a_0') ] - Q(s,a,\theta) \Vert^2\\
&\leq \mathbb E_{s' \sim \pi^*(.|s,a)}\Vert r(s,a)+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2},
\end{align*}

by the Cauchy-Schwarz inequality.

Thus, a good candidate for the onjective function to minimize in the training is eventually :

\begin{equation*}
\mathcal L(\theta) = \mathbb E_{s' \sim \pi^*(.|s,a)}\Vert r(s,a)+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) >= self.max_memory:
            self.memory = self.memory[1:]
        self.memory.append(m)

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None,:])[0])

    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            state, next_state, action, reward, game_over = self.memory.random_access()
            target_q[i] = self.model.predict(state[None,:])
            
            if game_over:
                target_q[i][action] = reward  
            else:  
                target_q[i][action] = reward + self.discount*max(self.model.predict(next_state[None,:])[0])        
            input_states[i] = state
            

        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)

        return l

    
    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(32 ,activation='relu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-7, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=200, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train200.mp4'))

Epoch 000/201 | Loss 0.0092 | Win/lose count 2.0/9.0 (-7.0)
Epoch 001/201 | Loss 0.0090 | Win/lose count 7.5/9.0 (-1.5)
Epoch 002/201 | Loss 0.0075 | Win/lose count 4.5/8.0 (-3.5)
Epoch 003/201 | Loss 0.0132 | Win/lose count 4.0/4.0 (0.0)
Epoch 004/201 | Loss 0.0155 | Win/lose count 7.5/5.0 (2.5)
Epoch 005/201 | Loss 0.0223 | Win/lose count 2.0/6.0 (-4.0)
Epoch 006/201 | Loss 0.0030 | Win/lose count 3.0/0 (3.0)
Epoch 007/201 | Loss 0.0196 | Win/lose count 4.0/7.0 (-3.0)
Epoch 008/201 | Loss 0.0084 | Win/lose count 2.5/7.0 (-4.5)
Epoch 009/201 | Loss 0.0051 | Win/lose count 1.0/3.0 (-2.0)
Epoch 010/201 | Loss 0.0582 | Win/lose count 5.0/3.0 (2.0)
Epoch 011/201 | Loss 0.0042 | Win/lose count 2.0/4.0 (-2.0)
Epoch 012/201 | Loss 0.0600 | Win/lose count 1.5/2.0 (-0.5)
Epoch 013/201 | Loss 0.0134 | Win/lose count 4.5/4.0 (0.5)
Epoch 014/201 | Loss 0.0032 | Win/lose count 4.0/6.0 (-2.0)
Epoch 015/201 | Loss 0.0370 | Win/lose count 2.5/1.0 (1.5)
Epoch 016/201 | Loss 0.0021 | Win/lose count 2.0

Epoch 139/201 | Loss 0.0617 | Win/lose count 14.5/1.0 (13.5)
Epoch 140/201 | Loss 0.0028 | Win/lose count 12.0/4.0 (8.0)
Epoch 141/201 | Loss 0.0023 | Win/lose count 7.5/2.0 (5.5)
Epoch 142/201 | Loss 0.0009 | Win/lose count 9.5/4.0 (5.5)
Epoch 143/201 | Loss 0.0486 | Win/lose count 8.0/2.0 (6.0)
Epoch 144/201 | Loss 0.0019 | Win/lose count 8.5/5.0 (3.5)
Epoch 145/201 | Loss 0.0033 | Win/lose count 11.5/10.0 (1.5)
Epoch 146/201 | Loss 0.0031 | Win/lose count 12.0/2.0 (10.0)
Epoch 147/201 | Loss 0.0029 | Win/lose count 16.0/2.0 (14.0)
Epoch 148/201 | Loss 0.0014 | Win/lose count 9.0/4.0 (5.0)
Epoch 149/201 | Loss 0.0040 | Win/lose count 5.5/3.0 (2.5)
Epoch 150/201 | Loss 0.0085 | Win/lose count 11.5/5.0 (6.5)
Epoch 151/201 | Loss 0.0009 | Win/lose count 15.0/2.0 (13.0)
Epoch 152/201 | Loss 0.0016 | Win/lose count 9.0/3.0 (6.0)
Epoch 153/201 | Loss 0.0039 | Win/lose count 10.5/3.0 (7.5)
Epoch 154/201 | Loss 0.0004 | Win/lose count 6.5/2.0 (4.5)
Epoch 155/201 | Loss 0.0056 | Win/lose coun

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(32, (2, 2), activation='relu',input_shape=(5,5,self.n_state)))
        model.add(Conv2D(16, (2, 2), activation='relu'))
        model.add(Dropout(0.1))
        model.add(Flatten())
        model.add(Dense(32 ,activation='relu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
epochs_train = 201
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=200, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train200.mp4'))

Epoch 000/201 | Loss 0.0044 | Win/lose count 4.5/7.0 (-2.5)
Epoch 001/201 | Loss 0.0144 | Win/lose count 5.5/11.0 (-5.5)
Epoch 002/201 | Loss 0.0073 | Win/lose count 3.0/8.0 (-5.0)
Epoch 003/201 | Loss 0.0044 | Win/lose count 3.0/4.0 (-1.0)
Epoch 004/201 | Loss 0.0025 | Win/lose count 1.5/4.0 (-2.5)
Epoch 005/201 | Loss 0.0056 | Win/lose count 5.0/3.0 (2.0)
Epoch 006/201 | Loss 0.0022 | Win/lose count 3.5/4.0 (-0.5)
Epoch 007/201 | Loss 0.0048 | Win/lose count 5.0/2.0 (3.0)
Epoch 008/201 | Loss 0.0080 | Win/lose count 3.0/6.0 (-3.0)
Epoch 009/201 | Loss 0.0079 | Win/lose count 4.5/8.0 (-3.5)
Epoch 010/201 | Loss 0.0009 | Win/lose count 0.5/1.0 (-0.5)
Epoch 011/201 | Loss 0.0062 | Win/lose count 9.0/1.0 (8.0)
Epoch 012/201 | Loss 0.0012 | Win/lose count 1.0/5.0 (-4.0)
Epoch 013/201 | Loss 0.0175 | Win/lose count 4.5/4.0 (0.5)
Epoch 014/201 | Loss 0.0176 | Win/lose count 5.5/4.0 (1.5)
Epoch 015/201 | Loss 0.0111 | Win/lose count 1.5/2.0 (-0.5)
Epoch 016/201 | Loss 0.0093 | Win/lose count

Epoch 138/201 | Loss 0.0488 | Win/lose count 8.0/6.0 (2.0)
Epoch 139/201 | Loss 0.0527 | Win/lose count 15.5/5.0 (10.5)
Epoch 140/201 | Loss 0.0060 | Win/lose count 5.5/1.0 (4.5)
Epoch 141/201 | Loss 0.0029 | Win/lose count 10.5/2.0 (8.5)
Epoch 142/201 | Loss 0.0040 | Win/lose count 11.0/2.0 (9.0)
Epoch 143/201 | Loss 0.0525 | Win/lose count 19.0/3.0 (16.0)
Epoch 144/201 | Loss 0.0538 | Win/lose count 5.0/3.0 (2.0)
Epoch 145/201 | Loss 0.0524 | Win/lose count 18.0/4.0 (14.0)
Epoch 146/201 | Loss 0.0057 | Win/lose count 24.0/3.0 (21.0)
Epoch 147/201 | Loss 0.0029 | Win/lose count 12.5/5.0 (7.5)
Epoch 148/201 | Loss 0.0044 | Win/lose count 17.0/0 (17.0)
Epoch 149/201 | Loss 0.0083 | Win/lose count 14.0/5.0 (9.0)
Epoch 150/201 | Loss 0.0058 | Win/lose count 19.5/4.0 (15.5)
Epoch 151/201 | Loss 0.0038 | Win/lose count 16.0/8.0 (8.0)
Epoch 152/201 | Loss 0.0528 | Win/lose count 9.5/4.0 (5.5)
Epoch 153/201 | Loss 0.0379 | Win/lose count 7.0/1.0 (6.0)
Epoch 154/201 | Loss 0.0032 | Win/lose co

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
epochs_test = 21
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=200, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=200, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 17.0/0.0. Average score (17.0)
Win/lose count 1.0/0.0. Average score (9.0)
Win/lose count 1.5/0.0. Average score (6.5)
Win/lose count 5.5/0.0. Average score (6.25)
Win/lose count 6.0/0.0. Average score (6.2)
Win/lose count 5.0/0.0. Average score (6.0)
Win/lose count 4.5/0.0. Average score (5.785714285714286)
Win/lose count 6.5/0.0. Average score (5.875)
Win/lose count 1.5/0.0. Average score (5.388888888888889)
Win/lose count 15.0/0.0. Average score (6.35)
Win/lose count 11.5/0.0. Average score (6.818181818181818)
Win/lose count 5.5/0.0. Average score (6.708333333333333)
Win/lose count 5.5/0.0. Average score (6.615384615384615)
Win/lose count 7.0/0.0. Average score (6.642857142857143)
Win/lose count 13.0/0.0. Average score (7.066666666666666)
Win/lose count 10.5/0.0. Average score (7.28125)
Win/lose count 22.0/0.0. Average score (8.147058823529411)
Win/lose count 3.0/0.0. Average score (7.861111111111111)
Win/lose count 3.5/0.0. Average score (7.6315789473

In [16]:
HTML(display_videos('cnn_test20.mp4'))

In [17]:
HTML(display_videos('fc_test20.mp4'))

We can observe that the agent trained by the cnn have better performances than the one trained by the fully connected network.
However, a major drawback remaining is the tendency of the game to block quickly far before to reach the time $T$.
Such an issue can be tackle by forcing a better exploration, by adding a malus when a state have already been visited.

Changing the temperature don't solve this general behaviour, but we can observe that the higher the temperature is set, the higher final scores we get. It is explained by the fact that there is a higher bonus concentration in the the space when the temperature is increased.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
def train_explore(agent,env,epoch,prefix=''):
    score = 0
    loss = 0
    for e in range(epoch):
        state = env.reset()
        game_over = False
        win = 0
        lose = 0
        while not game_over:
            agent.set_epsilon(agent.epsilon*0.95)
            action = agent.act(state,train=True)
            prev_state = state
            state, reward, game_over = env.act(action,train=True)
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        if e % 10 == 0:
            env.draw(prefix+str(e))
        score += win-lose
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    print('Average Score = ', score/epoch)

In [19]:

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        self.x = 0
        self.y = 1

        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:] = 256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train = True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y] - int(train)*self.malus_position[self.x,self.y]
        self.malus_position[self.x,self.y] = 0.1
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]


        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state

In [20]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=200, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore200.mp4'))

Epoch 000/201 | Loss 0.0110 | Win/lose count 6.0/22.700000000000067 (-16.700000000000067)
Epoch 001/201 | Loss 0.0058 | Win/lose count 2.5/21.50000000000003 (-19.00000000000003)
Epoch 002/201 | Loss 0.0112 | Win/lose count 9.5/18.199999999999996 (-8.699999999999996)
Epoch 003/201 | Loss 0.0182 | Win/lose count 0.5/22.9 (-22.4)
Epoch 004/201 | Loss 0.0139 | Win/lose count 9.0/19.500000000000014 (-10.500000000000014)
Epoch 005/201 | Loss 0.0071 | Win/lose count 4.0/18.9 (-14.899999999999999)
Epoch 006/201 | Loss 0.0395 | Win/lose count 8.5/19.69999999999999 (-11.199999999999989)
Epoch 007/201 | Loss 0.0184 | Win/lose count 11.5/23.700000000000024 (-12.200000000000024)
Epoch 008/201 | Loss 0.0260 | Win/lose count 12.5/18.500000000000004 (-6.0000000000000036)
Epoch 009/201 | Loss 0.0056 | Win/lose count 10.5/14.099999999999966 (-3.599999999999966)
Epoch 010/201 | Loss 0.0168 | Win/lose count 8.5/20.700000000000024 (-12.200000000000024)
Epoch 011/201 | Loss 0.0148 | Win/lose count 7.5/20.70

Epoch 092/201 | Loss 0.0281 | Win/lose count 21.0/17.199999999999992 (3.800000000000008)
Epoch 093/201 | Loss 0.0302 | Win/lose count 20.0/16.499999999999975 (3.500000000000025)
Epoch 094/201 | Loss 0.0133 | Win/lose count 12.5/17.499999999999982 (-4.999999999999982)
Epoch 095/201 | Loss 0.0214 | Win/lose count 17.5/12.599999999999975 (4.900000000000025)
Epoch 096/201 | Loss 0.0468 | Win/lose count 23.0/17.19999999999999 (5.800000000000011)
Epoch 097/201 | Loss 0.0632 | Win/lose count 16.5/13.499999999999968 (3.000000000000032)
Epoch 098/201 | Loss 0.0274 | Win/lose count 17.0/18.19999999999998 (-1.1999999999999815)
Epoch 099/201 | Loss 0.0264 | Win/lose count 21.5/17.29999999999999 (4.20000000000001)
Epoch 100/201 | Loss 0.0321 | Win/lose count 15.0/15.199999999999962 (-0.19999999999996199)
Epoch 101/201 | Loss 0.0226 | Win/lose count 19.0/12.599999999999977 (6.4000000000000234)
Epoch 102/201 | Loss 0.0278 | Win/lose count 19.0/15.799999999999969 (3.2000000000000313)
Epoch 103/201 | L

Epoch 184/201 | Loss 0.0295 | Win/lose count 24.5/12.999999999999975 (11.500000000000025)
Epoch 185/201 | Loss 0.0450 | Win/lose count 24.5/13.799999999999974 (10.700000000000026)
Epoch 186/201 | Loss 0.0485 | Win/lose count 16.0/15.69999999999997 (0.30000000000002913)
Epoch 187/201 | Loss 0.0079 | Win/lose count 13.0/17.19999999999998 (-4.1999999999999815)
Epoch 188/201 | Loss 0.0457 | Win/lose count 21.5/15.59999999999997 (5.9000000000000306)
Epoch 189/201 | Loss 0.0219 | Win/lose count 15.0/16.199999999999967 (-1.1999999999999673)
Epoch 190/201 | Loss 0.0203 | Win/lose count 14.5/16.59999999999997 (-2.0999999999999694)
Epoch 191/201 | Loss 0.0237 | Win/lose count 9.0/20.40000000000001 (-11.40000000000001)
Epoch 192/201 | Loss 0.0181 | Win/lose count 17.5/18.300000000000008 (-0.8000000000000078)
Epoch 193/201 | Loss 0.0087 | Win/lose count 10.0/14.499999999999964 (-4.4999999999999645)
Epoch 194/201 | Loss 0.0317 | Win/lose count 13.5/18.59999999999998 (-5.09999999999998)
Epoch 195/20

In [21]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore20.mp4'))

Win/lose count 7.5/17.799999999999983. Average score (-10.299999999999983)
Win/lose count 9.5/18.5. Average score (-9.649999999999991)
Win/lose count 5.0/21.10000000000003. Average score (-11.800000000000004)
Win/lose count 4.0/19.300000000000004. Average score (-12.675000000000004)
Win/lose count 3.5/19.200000000000003. Average score (-13.280000000000005)
Win/lose count 18.5/17.799999999999994. Average score (-10.950000000000003)
Win/lose count 20.0/15.199999999999964. Average score (-8.699999999999998)
Win/lose count 14.0/18.299999999999997. Average score (-8.149999999999997)
Win/lose count 7.0/17.69999999999998. Average score (-8.433333333333328)
Win/lose count 8.0/18.799999999999997. Average score (-8.669999999999995)
Win/lose count 4.5/18.599999999999998. Average score (-9.163636363636359)
Win/lose count 6.5/18.29999999999999. Average score (-9.383333333333328)
Win/lose count 20.0/17.8. Average score (-8.492307692307687)
Win/lose count 4.5/18.599999999999994. Average score (-8.892

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***